In [43]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('train.csv')
df.head()

,date,sr,fact,enc_hub
0,2021-12-03,1,2,78
1,2022-06-04,27,79,28
2,2021-03-22,9380,11002,104
3,2021-07-31,294,308,70
4,2022-06-04,2,306,92


In [64]:
def prep_data(df, lag_counts:(int,int)):
    df.date = df.date.astype(np.datetime64)
    df.fact = df.fact.astype(np.float64)
    df.sr = df.sr.astype(np.float64)
    data = df.set_index(['enc_hub', 'date'])
    data['y'] = data.groupby(level=0)['fact'].shift(-1) 
    
    for sr_lag in range(1, lag_counts[0]+1):
        data[f'sr_{sr_lag}'] = data.groupby(level=0)['fact'].shift(sr_lag)
    
    for targ_lag in range(1, lag_counts[1]+1):
        data[f'targ_{targ_lag}'] = data.groupby(level=0)['fact'].shift(targ_lag)
    
    return data.dropna()

In [65]:
prep_data(df, [2, 2])

,,sr,fact,y,sr_1,sr_2,targ_1,targ_2
enc_hub,date,,,,,,,
65,2022-03-03,10837.0,11988.0,8781.0,14873.0,10171.0,14873.0,10171.0
70,2021-06-07,266.0,239.0,190.0,135.0,308.0,135.0,308.0
31,2021-12-03,4911.0,5462.0,1567.0,2855.0,4898.0,2855.0,4898.0
104,2022-03-03,15255.0,18999.0,9967.0,21017.0,11002.0,21017.0,11002.0
79,2022-03-03,1032.0,1358.0,195.0,124.0,1126.0,124.0,1126.0
...,...,...,...,...,...,...,...,...
65,2021-05-26,13353.0,13074.0,3895.0,10272.0,8804.0,10272.0,8804.0
63,2021-06-21,17684.0,18697.0,20343.0,8663.0,19432.0,8663.0,19432.0
25,2021-06-20,1367.0,1536.0,1607.0,5227.0,3531.0,5227.0,3531.0


In [33]:
from linearmodels import RandomEffects
results = RandomEffects('fact ~ Literacy + np.log(Pop1831)', data=dat).fit()

sr  fact
date       enc_hub           
2021-01-02 63       302  6200
           94         1     7
           77         5    71
           128       21   106
           135       22   144

In [34]:
y = X.fact.shift(-1)
y.head()

date        enc_hub
2021-01-02  63           7.0
            94          71.0
            77         106.0
            128        144.0
            135         10.0
Name: fact, dtype: float64

In [9]:
from sklearn.model_selection import TimeSeriesSplit

tcsv = TimeSeriesSplit(n_splits=10, test_size=10)

In [10]:
tcsv.split(df)

<generator object TimeSeriesSplit.split at 0x7f946178d3f0>

In [ ]:
for train_index, test_index in tscv.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]